In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [2]:
training_set=tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 70295 files belonging to 38 classes.


In [3]:
validation_set=tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

Found 17572 files belonging to 38 classes.


In [4]:
training_set

<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 38), dtype=tf.float32, name=None))>

In [5]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications.vgg16 import VGG16

In [6]:
from tensorflow.keras.callbacks import Callback
from timeit import default_timer as timer

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

In [7]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

early_stopping = EarlyStopping(
                                patience=5,
                                min_delta = 0.01,
                                verbose=1,
                                mode = 'min',
                                monitor='val_loss')

reduce_learning_rate = ReduceLROnPlateau(
                                    monitor="val_loss",
                                    patience=3,
                                    episilon= 0.01,
                                    factor=0.1,
                                    cooldown = 4,
                                    verbose=1)

time_callback = TimingCallback()

In [8]:
n_class=38
base_model = VGG16(weights='imagenet', include_top=False)
for layer in base_model.layers: 
    layer.trainable = False
model=Sequential()

In [9]:
model.add(base_model)
model.add(GlobalAveragePooling2D()) 
model.add(Dense(1024,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(n_class, activation='softmax'))

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              525312    
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                        

In [12]:
batch_size = 128
epochs = 20 
model_history = model.fit(
            training_set,
            validation_data =validation_set,
            epochs = epochs,
            callbacks = [
                         reduce_learning_rate,
                         early_stopping,
                         time_callback
                        ],
            verbose=True)

Epoch 1/20
2197/2197 [==============================] - 1003s 456ms/step - loss: 0.6433 - acc: 0.8111 - val_loss: 0.2984 - val_acc: 0.9010 - lr: 0.0010
Epoch 2/20
2197/2197 [==============================] - 1121s 510ms/step - loss: 0.3517 - acc: 0.8893 - val_loss: 0.2692 - val_acc: 0.9160 - lr: 0.0010
Epoch 3/20
2197/2197 [==============================] - 1025s 467ms/step - loss: 0.3040 - acc: 0.9066 - val_loss: 0.2573 - val_acc: 0.9214 - lr: 0.0010
Epoch 4/20
2197/2197 [==============================] - 1004s 457ms/step - loss: 0.2739 - acc: 0.9162 - val_loss: 0.2478 - val_acc: 0.9226 - lr: 0.0010
Epoch 5/20
2197/2197 [==============================] - 1051s 478ms/step - loss: 0.2552 - acc: 0.9226 - val_loss: 0.2534 - val_acc: 0.9270 - lr: 0.0010
Epoch 6/20
2197/2197 [==============================] - 1000s 455ms/step - loss: 0.2444 - acc: 0.9291 - val_loss: 0.2586 - val_acc: 0.9280 - lr: 0.0010
Epoch 7/20
2197/2197 [==============================] - ETA: 0s - loss: 0.2388 - acc: 0.